## House Prices Model Estimation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [7]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
housing_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [8]:
housing_reg_df =  housing_df.copy()
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.centralair, 
                                                            prefix="centralair", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.mszoning, 
                                                            prefix="mszoning", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.bldgtype, 
                                                            prefix="bldgtype", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.exterqual, 
                                                            prefix="exterqual", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.bsmtqual, 
                                                            prefix="bsmtqual", drop_first=True)], axis=1)
housing_reg_df = pd.concat([housing_reg_df,pd.get_dummies(housing_reg_df.salecondition, 
                                                            prefix="salecondition", drop_first=True)], axis=1)

In [9]:
dummy_column_names = list(pd.get_dummies(housing_reg_df.centralair, prefix="centralair", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(housing_reg_df.mszoning, prefix="mszoning", 
                drop_first=True).columns)+ list(pd.get_dummies(housing_reg_df.bldgtype, prefix="bldgtype", 
                drop_first=True).columns)+ list(pd.get_dummies(housing_reg_df.exterqual, prefix="exterqual", 
                drop_first=True).columns)+ list(pd.get_dummies(housing_reg_df.bsmtqual, prefix="bsmtqual", 
                drop_first=True).columns)+ list(pd.get_dummies(housing_reg_df.salecondition, prefix="salecondition", 
                drop_first=True).columns)

In [10]:
X = housing_reg_df[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'garagearea', 'firstflrsf'] 
                   + dummy_column_names]
Y = housing_reg_df['saleprice']

In [11]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.805
Method:                 Least Squares   F-statistic:                     232.8
Date:                Sun, 23 Jun 2019   Prob (F-statistic):               0.00
Time:                        18:24:31   Log-Likelihood:                -17337.
No. Observations:                1460   AIC:                         3.473e+04
Df Residuals:                    1433   BIC:                         3.487e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -2168.7776   1.55e+04     -0.140      0.889   -3.25e+04    2.82e+04
overallqual            1.533e+04   1215.885     12.609      0.000    1.29e+04    1.77e+04
grlivarea                46.6127      2.533     18.402      0.000      41.644      51.582
garagecars             1.348e+04   2841.606      4.745      0.000    7909.485    1.91e+04
totalbsmtsf              29.1973      4.239      6.888      0.000      20.882      37.513
garagearea               -0.8777      9.758     -0.090      0.928     -20.020      18.265
firstflrsf               -4.1746      5.030     -0.830      0.407     -14.041       5.691
centralair_Y           1.433e+04   4257.582      3.365      0.001    5974.934    2.27e+04
mszoning_FV            2.375e+04   1.28e+04      1.851      0.064   -1423.693    4.89e+04
mszoning_RH             2.28e+04   1.45e+04      1.571      0.116   -5675.126    5.13e+04
mszoning_RL            2.676e+04    1.2e+04      2.235      0.026    3276.117    5.03e+04
mszoning_RM            1.293e+04    1.2e+04      1.073      0.283   -1.07e+04    3.66e+04
bldgtype_2fmCon       -6925.9315   6731.875     -1.029      0.304   -2.01e+04    6279.454
bldgtype_Duplex       -2.762e+04   5514.705     -5.008      0.000   -3.84e+04   -1.68e+04
bldgtype_Twnhs        -1.583e+04   5816.846     -2.721      0.007   -2.72e+04   -4417.086
bldgtype_TwnhsE       -1.247e+04   3838.718     -3.247      0.001      -2e+04   -4934.994
exterqual_Fa          -4.285e+04   1.24e+04     -3.460      0.001   -6.71e+04   -1.86e+04
exterqual_Gd           -3.68e+04   6074.745     -6.058      0.000   -4.87e+04   -2.49e+04
exterqual_TA          -4.959e+04   6786.021     -7.308      0.000   -6.29e+04   -3.63e+04
bsmtqual_Fa           -4.271e+04   7226.473     -5.910      0.000   -5.69e+04   -2.85e+04
bsmtqual_Gd           -3.101e+04   3808.919     -8.141      0.000   -3.85e+04   -2.35e+04
bsmtqual_TA           -3.712e+04   4177.552     -8.887      0.000   -4.53e+04   -2.89e+04
salecondition_AdjLand  1.673e+04   1.83e+04      0.916      0.360   -1.91e+04    5.26e+04
salecondition_Alloca   1.531e+04   1.13e+04      1.350      0.177   -6931.253    3.76e+04
salecondition_Family  -3551.2137   8671.374     -0.410      0.682   -2.06e+04    1.35e+04
salecondition_Normal   8511.4287   3723.582      2.286      0.022    1207.172    1.58e+04
salecondition_Partial  1.606e+04   5175.225      3.103      0.002    5904.392    2.62e+04
==============================================================================
Omnibus:                      760.925   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           115462.645
Skew:                          -1.367   Prob(JB):                         0.00
Kurtosis:                      46.480   Cond. No.                  

The statistically significant variables are overallqual, grlivarea, garagecars, totalbsmtsf, centralair_Y, bldgtype_Duplex, bldgtype_Twnhs, bldgtype_TwnhsE, exterqual_Gd, exterqual_TA, bsmtqual_Fa, bsmtqual_Gd, bsmtqual_TA, salecondition_Normal, salecondition_Partial. All the other variables are statistically insignificant and hence their coefficients are statistically zero. According to the estimation results:

1 point increase in overallqual results in \$15330 increase in sale price.
1 point increase in grlivarea results in \$46.6127 increase in sale price.
1 point increase in garagecars results in \$13480 increase in sale price.
1 point increase in totalbsmtsf results in \$29.1973 increase in sale price.
1 point increase in centralair_Y results in \$14334 increase in sale price.
1 point increase in bldgtype_Duplex results in \$27620 decrease in sale price.
1 point increase in bldgtype_Twnhs results in \$15834 decrease in sale price.
1 point increase in bldgtype_TwnhsE results in \$12470 decrease in sale price.
1 point increase in exterqual_Gd results in \$36800 decrease in sale price.
1 point increase in exterqual_TA results in \$49590 decrease in sale price.
1 point increase in bsmtqual_Fa results in \$42710 decrease in sale price.
1 point increase in bsmtqual_Gd results in \$31010 decrease in sale price.
1 point increase in bsmtqual_TA results in \$37120 decrease in sale price.
1 point increase in salecondition_Normal results in \$8511.4287 increase in sale price.
1 point increase in salecondition_Partial results in \$16060 increase in sale price.

- excluding the statistically insignificant variables from the model and re-estimating it:

In [15]:
# Y is the target variable
Y = housing_reg_df['saleprice']
# X is the feature set
X = housing_reg_df[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf', 'centralair_Y', 'bldgtype_Duplex',
                     'bldgtype_Twnhs','bldgtype_TwnhsE','exterqual_Gd','exterqual_TA','bsmtqual_Fa','bsmtqual_Gd',
                     'bsmtqual_TA','salecondition_Normal','salecondition_Partial']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     391.1
Date:                Sun, 23 Jun 2019   Prob (F-statistic):               0.00
Time:                        18:48:34   Log-Likelihood:                -17360.
No. Observations:                1460   AIC:                         3.475e+04
Df Residuals:                    1444   BIC:                         3.484e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -7141.4517   9585.443     -0.745      0.456   -2.59e+04    1.17e+04
overallqual            1.671e+04   1174.129     14.231      0.000    1.44e+04     1.9e+04
grlivarea                45.0136      2.375     18.949      0.000      40.354      49.673
garagecars             1.345e+04   1675.413      8.025      0.000    1.02e+04    1.67e+04
totalbsmtsf              29.1075      2.695     10.802      0.000      23.822      34.393
centralair_Y           2.132e+04   4067.847      5.241      0.000    1.33e+04    2.93e+04
bldgtype_Duplex       -2.464e+04   5302.808     -4.646      0.000    -3.5e+04   -1.42e+04
bldgtype_Twnhs        -2.148e+04   5633.936     -3.813      0.000   -3.25e+04   -1.04e+04
bldgtype_TwnhsE       -1.668e+04   3696.526     -4.512      0.000   -2.39e+04   -9427.665
exterqual_Gd          -2.356e+04   5225.885     -4.508      0.000   -3.38e+04   -1.33e+04
exterqual_TA          -3.391e+04   5528.691     -6.133      0.000   -4.48e+04   -2.31e+04
bsmtqual_Fa           -4.927e+04   7114.394     -6.926      0.000   -6.32e+04   -3.53e+04
bsmtqual_Gd           -3.279e+04   3605.426     -9.095      0.000   -3.99e+04   -2.57e+04
bsmtqual_TA           -4.124e+04   3986.902    -10.344      0.000   -4.91e+04   -3.34e+04
salecondition_Normal   8698.1326   3250.440      2.676      0.008    2322.044    1.51e+04
salecondition_Partial  1.688e+04   4829.464      3.494      0.000    7401.971    2.63e+04
==============================================================================
Omnibus:                      738.694   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           101640.888
Skew:                          -1.318   Prob(JB):                         0.00
Kurtosis:                      43.791   Cond. No.                     2.31e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.31e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- All terms remained statistically significant.
- Coefficient magnitudes also remained similar to the previous estimation model.
- The most prominent factors in determing sale price is externale quality and basement quality. 